In [2]:
#input_path ="C:\\Users\\arpit\\Downloads\\dyson\\input_files"

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_timestamp
from pyspark.sql.functions import *

import pyarrow as pa

import yaml

# Create a Spark session
spark = SparkSession.builder.appName("ETL").getOrCreate()


In [28]:
# Function to read the configuration from config.yaml
def read_config():
    try:
        with open('config.yaml', 'r') as config_file:
            config = yaml.safe_load(config_file)
        return config
    except FileNotFoundError:
        print("Error: config.yaml file not found.")
        return None
    except yaml.YAMLError as e:
        print(f"Error loading config.yaml: {e}")
        return None

if __name__ == "__main__":
    # Read the configuration from config.yaml
    config = read_config()

    if config:
        # Access the input_path from the config
        input_path = config.get('input_path')
        output_path = config.get('output_path')

        # Print the input_path (you can use it in your PySpark pipeline)
        print(f"Input Path: {input_path}")
        print(f"Output Path: {output_path}")
    else:
        print("Configuration not loaded. Please check the YAML file and its location.")


Input Path: C:\Users\arpit\Downloads\dyson\source
Output Path: C:\Users\arpit\Downloads\dyson\target


In [17]:
# Read the CSV file into a DataFrame
df_char = spark.read.csv(f"{input_path}\characters.csv", header=True, inferSchema=True)
df_char.show(5)

+-----------+---------------+
|characterID|           name|
+-----------+---------------+
|    1009220|Captain America|
|    1010740| Winter Soldier|
|    1009471|      Nick Fury|
|    1009552|   S.H.I.E.L.D.|
|    1009228|  Sharon Carter|
+-----------+---------------+
only showing top 5 rows



In [19]:
# Read the CSV file into a DataFrame
df_stats = spark.read.csv(f"{input_path}\characters_stats.csv", header=True, inferSchema=True)
df_stats = df_stats.withColumnRenamed("Name","name")
df_stats.show(5)

+-----------+---------+------------+--------+-----+----------+-----+------+-----+
|       name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat|Total|
+-----------+---------+------------+--------+-----+----------+-----+------+-----+
|    3-D Man|     good|          50|      31|   43|        32|   25|    52|  233|
|     A-Bomb|     good|          38|     100|   17|        80|   17|    64|  316|
| Abe Sapien|     good|          88|      14|   35|        42|   35|    85|  299|
|   Abin Sur|     good|          50|      90|   53|        64|   84|    65|  406|
|Abomination|      bad|          63|      80|   53|        90|   55|    95|  436|
+-----------+---------+------------+--------+-----+----------+-----+------+-----+
only showing top 5 rows



In [27]:
df_char_stats = df_char.join(df_stats, on="name", how="inner")
# Add audit columns
df_char_stats = df_char_stats.withColumn("batch_id", lit("101"))
df_char_stats = df_char_stats.withColumn("load_date", current_timestamp().cast("string"))
df_char_stats.show(5)


+---------------+-----------+---------+------------+--------+-----+----------+-----+------+-----+--------+--------------------+
|           name|characterID|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat|Total|batch_id|           load_date|
+---------------+-----------+---------+------------+--------+-----+----------+-----+------+-----+--------+--------------------+
|Captain America|    1009220|     good|          63|      19|   35|        56|   46|   100|  319|     101|2023-09-15 01:25:...|
| Winter Soldier|    1010740|     good|          56|      32|   35|        65|   60|    84|  332|     101|2023-09-15 01:25:...|
|      Nick Fury|    1009471|     good|          75|      11|   23|        42|   25|   100|  276|     101|2023-09-15 01:25:...|
|       Punisher|    1009515|     good|          50|      16|   23|        28|   22|   100|  239|     101|2023-09-15 01:25:...|
|      Red Skull|    1009535|      bad|          75|      10|   12|        14|   19|    80|  210|     10

In [37]:
# Save the DataFrame to a Parquet file
dfp_char_stats = df_char_stats.toPandas()
try:
    dfp_char_stats.to_parquet(f"{output_path}\char_stats.parquet", index=False, engine="pyarrow")
    # Print a message to confirm the file has been saved
    print(f"DataFrame saved to Parquet file: {output_path}")
except Exception as e:
    print(f"Error writing DataFrame to Parquet: {str(e)}")


DataFrame saved to Parquet file: C:\Users\arpit\Downloads\dyson\target
